PROJECT 3

Part 1: Prediction quality vs feature selection

In [2]:
from sklearn.linear_model import LassoCV, Lasso

Hyperparameter $\lambda$ are instead $\alpha$ in LassoCV

`alpha_` contains hyperparameter that achieved lowest MSE

`alphas_` contains all tested hyperparameters

Coeffs of `Lasso` can be found in `coef_`

`mse_path_` a `(len(alphas_) x n_folds)` matrix storing test error on each fold

Vectors **e** and **s** can easily be computed from `mse_path_`

In [ ]:
# EXAMPLE CODE FROM THESIS

cv_mean = np.mean(lasso.mse_path_, axis=1)
cv_std = np.std(lasso.mse_path_, axis=1)
idx_min_mean = np.argmin(cv_mean)
idx_alpha = np.where(
    (cv_mean <= cv_mean[idx_min_mean] + cv_std[idx_min_mean] / np.sqrt(n_folds)) &
    (cv_mean >= cv_mean[idx_min_mean])
)[0][0]
alpha_1se = lasso.alphas_[idx_alpha]